In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def visualize_and_track_roi_slices(directory, output_csv):
    """Visualize ROI slices, track their indices, and save the data."""
    records = []

    for filename in os.listdir(directory):
        if filename.startswith('mask_'):
            file_path = os.path.join(directory, filename)
            try:
                image = nib.load(file_path)
                data = image.get_fdata()
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                continue

            roi_slices = []
            slice_visibilities = []

            for i in range(data.shape[-1]):
                slice_data = data[:, :, i]
                slice_sum = np.sum(slice_data)
                if slice_sum > 0:
                    roi_slices.append(i)
                    slice_visibilities.append(slice_sum)

            if len(roi_slices) > 5:
                top_slices = sorted(zip(roi_slices, slice_visibilities), key=lambda x: x[1], reverse=True)[:5]
                roi_slices = [x[0] for x in top_slices]
                max_visibility_slice = roi_slices[0]
            elif len(roi_slices) == 0:
                max_visibility_slice = None
            else:
                max_visibility_slice = roi_slices[np.argmax(slice_visibilities)]






            records.append({
                'Image Name': filename,
                'ROI Slices': roi_slices,
                'Max Visibility Slice': max_visibility_slice
            })

            print(f"Processed {filename}: {len(roi_slices)} slices with ROI, max visibility slice {max_visibility_slice}")

    # Create a DataFrame from the records
    df = pd.DataFrame(records)
    df.to_csv(output_csv, index=False)
    print(f"Saved ROI slice data to {output_csv}")

processed_roi_directory = "/content/drive/MyDrive/OriginalData/Flair_ROI"
output_csv_path = "/content/drive/MyDrive/newData/roi_slice_data.csv"

visualize_and_track_roi_slices(processed_roi_directory, output_csv_path)



Processed mask_sub-00001_acq-T2sel_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 138
Processed mask_sub-00003_acq-T2sel_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 161
Processed mask_sub-00004_acq-T2sel_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 150
Processed mask_sub-00006_acq-tse3dvfl_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 163
Processed mask_sub-00009_acq-T2sel_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 131
Processed mask_sub-00010_acq-T2sel_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 145
Processed mask_sub-00014_acq-tse3dvfl_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 118
Processed mask_sub-00015_acq-T2sel_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 133
Processed mask_sub-00016_acq-tse3dvfl_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 162
Processed mask_sub-00018_acq-corhipp4mm_FLAIR_roi.nii.gz: 5 slices with ROI, max visibility slice 12
Processed mask_su

In [ ]:
import os
import json
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file with the most visible slices information
output_csv_path = "/content/drive/MyDrive/newData/roi_slice_data.csv"
roi_data = pd.read_csv(output_csv_path)

# Define paths
mri_dir = '/content/drive/MyDrive/OriginalData/Epileptic'
roi_dir = '/content/drive/MyDrive/OriginalData/Flair_ROI'
base_dir = '/content/drive/MyDrive/newData/nnUNet_raw'
dataset_dir = os.path.join(base_dir, 'Dataset001_CustomDataset')
images_tr_dir = os.path.join(dataset_dir, 'imagesTr')
labels_tr_dir = os.path.join(dataset_dir, 'labelsTr')
images_2d_dir = os.path.join(dataset_dir, 'imagesTr_2D')
labels_2d_dir = os.path.join(dataset_dir, 'labelsTr_2D')

# Create directories if they don't exist
os.makedirs(images_tr_dir, exist_ok=True)
os.makedirs(labels_tr_dir, exist_ok=True)
os.makedirs(images_2d_dir, exist_ok=True)
os.makedirs(labels_2d_dir, exist_ok=True)

# Initialize dataset description
dataset_description = {
    "channel_names": {"0": "FLAIR"},
    "labels": {"background": 0, "ROI": 1},
    "numTraining": len(roi_data),
    "file_ending": ".nii.gz"
}

# Process each entry in the CSV
for idx, row in roi_data.iterrows():
    mri_file = row['Image Name'].replace('mask_', 'preprocessed_')
    roi_file = row['Image Name']
    slices = eval(row['ROI Slices'])
    case_id = f"case_{idx:04d}"

    try:
        # Load MRI and ROI files
        mri_img = nib.load(os.path.join(mri_dir, mri_file))
        roi_img = nib.load(os.path.join(roi_dir, roi_file))

        mri_data = mri_img.get_fdata()
        roi_data = roi_img.get_fdata()

        # Extract specified slices
        mri_slices = mri_data[:, :, slices]
        roi_slices = roi_data[:, :, slices]

        # Save new MRI and ROI files
        new_mri_img = nib.Nifti1Image(mri_slices, mri_img.affine)
        new_roi_img = nib.Nifti1Image(roi_slices, roi_img.affine)

        new_mri_file = os.path.join(images_tr_dir, f"{case_id}_0000.nii.gz")
        new_roi_file = os.path.join(labels_tr_dir, f"{case_id}.nii.gz")

        nib.save(new_mri_img, new_mri_file)
        nib.save(new_roi_img, new_roi_file)

        # Save 2D images of the slices
        for i, slice_idx in enumerate(slices):
            mri_slice_img = mri_slices[:, :, i]
            roi_slice_img = roi_slices[:, :, i]

            mri_img_path = os.path.join(images_2d_dir, f"{case_id}_slice_{slice_idx:04d}.png")
            roi_img_path = os.path.join(labels_2d_dir, f"{case_id}_slice_{slice_idx:04d}.png")

            plt.imsave(mri_img_path, mri_slice_img.T, cmap='gray')
            plt.imsave(roi_img_path, roi_slice_img.T, cmap='gray')

        print(f"Processed {mri_file} and {roi_file}: saved slices {slices}")

    except Exception as e:
        print(f"Error processing {mri_file} and {roi_file}: {e}")

# Save dataset description
dataset_json_path = os.path.join(dataset_dir, 'dataset.json')
with open(dataset_json_path, 'w') as f:
    json.dump(dataset_description, f, indent=4)

print("Dataset creation complete")


Processed preprocessed_sub-00001_acq-T2sel_FLAIR_roi.nii.gz and mask_sub-00001_acq-T2sel_FLAIR_roi.nii.gz: saved slices [138, 139, 137, 140, 136]
Processed preprocessed_sub-00003_acq-T2sel_FLAIR_roi.nii.gz and mask_sub-00003_acq-T2sel_FLAIR_roi.nii.gz: saved slices [161, 160, 157, 158, 159]
Processed preprocessed_sub-00004_acq-T2sel_FLAIR_roi.nii.gz and mask_sub-00004_acq-T2sel_FLAIR_roi.nii.gz: saved slices [150, 149, 151, 148, 147]
Processed preprocessed_sub-00006_acq-tse3dvfl_FLAIR_roi.nii.gz and mask_sub-00006_acq-tse3dvfl_FLAIR_roi.nii.gz: saved slices [163, 165, 166, 164, 162]
Processed preprocessed_sub-00009_acq-T2sel_FLAIR_roi.nii.gz and mask_sub-00009_acq-T2sel_FLAIR_roi.nii.gz: saved slices [131, 130, 132, 133, 129]
Processed preprocessed_sub-00010_acq-T2sel_FLAIR_roi.nii.gz and mask_sub-00010_acq-T2sel_FLAIR_roi.nii.gz: saved slices [145, 144, 143, 142, 147]
Processed preprocessed_sub-00014_acq-tse3dvfl_FLAIR_roi.nii.gz and mask_sub-00014_acq-tse3dvfl_FLAIR_roi.nii.gz: saved